In [1]:
import sqlite3
import pandas as pd

## Setup dbase in RAM

In [2]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [3]:
sql_dump = open(r"db_sql\db_dmp.sql","r")
sql_statements = (sql_dump
                    .read()
                    .split(";"))
sql_dump.close()
                
for st in sql_statements:
    try:
        cur.execute(st)
    except sqlite3.Error as e:
        print(e)              

## Check DB (get list of tables)

In [4]:
pd.read_sql_query(
    """ SELECT name
        FROM sqlite_master 
        WHERE type ='table'
        AND name NOT LIKE 'sqlite_%'; """,
        con)

,name
0,medicine
1,diagnosis
2,job
3,workers
4,contacts


## Run queries in SQL
a. Напишите запрос,который выводит название позиций(без повторений), хотя бы один представитель которой получает зарплату в промежутке 400-600

In [5]:
pd.read_sql_query(
    """ SELECT DISTINCT(POSITION)
        FROM job
        WHERE SALARY BETWEEN 400 AND 600; """,
        con)

,POSITION
0,C# DEVELOPER
1,AUTOMATION QA ENGINEER
2,JAVA DEVELOPER


b. Напишите запрос, который выполняет вывод списка офисов, расположенных в SEBEWAING и ELECTRON, количество рабочих и средней зарплаты рабочих в каждом офисе

In [6]:
pd.read_sql_query(
    """ SELECT OFFICE, count(WORKER_ID), avg(SALARY) as MEAN_SALARY
        FROM job
        WHERE OFFICE_CITY IN ("SEBEWAING", "ELECTRON")
        GROUP BY OFFICE; """,
        con)

,OFFICE,count(WORKER_ID),MEAN_SALARY
0,TIME IS MONEY,7,725.958571


c. Напишите запрос, который выводит название профессий (без повторений), все представители которой получают зарплату в промежутке 400-700

In [7]:
pd.read_sql_query(
    """ SELECT DISTINCT(POSITION)
        FROM job
        WHERE POSITION NOT IN
            (SELECT POSITION
            FROM job
            WHERE SALARY NOT BETWEEN 400 AND 700); """,
        con)

,POSITION
0,AUTOMATION QA ENGINEER
1,JAVA DEVELOPER


d. Напишите запрос, который выводит среднюю зарплату работников, которые старше 30 лет, группируя по полу и возрасту, но при выводе увеличивающий данные о величине зарплаты на 20%

In [8]:
pd.read_sql_query(
    """ SELECT  substr(date(),1,4) - 19||substr(DOB,8,2) AS AGE,
                GENDER,
                AVG(SALARY) * 1.2 AS AVG_SALARY_UP
        FROM workers t1
        INNER JOIN job t2
        ON t1.WORKER_ID = t2.WORKER_ID
        WHERE AGE > 30
        GROUP BY AGE, GENDER; """,
        con)

,AGE,GENDER,AVG_SALARY_UP
0,34,F,1052.370
1,34,M,691.544
2,35,F,1175.004
3,39,F,665.244


e. Напишите запрос, который выведет накопленную зарплату по месяцам для Николы Теслы начиная с его приему на работу до сегодняшнего дня.

In [9]:
pd.read_sql_query("""
WITH RECURSIVE dates AS
  (SELECT min(CASE substr(FROM_DATE, 4, 3)
                  WHEN 'Jan' THEN 20||substr(FROM_DATE, 8, 2)||'-01'
                  WHEN 'Feb' THEN 20||substr(FROM_DATE, 8, 2)||'-02'
                  WHEN 'Mar' THEN 20||substr(FROM_DATE, 8, 2)||'-03'
                  WHEN 'Apr' THEN 20||substr(FROM_DATE, 8, 2)||'-04'
                  WHEN 'May' THEN 20||substr(FROM_DATE, 8, 2)||'-05'
                  WHEN 'Jun' THEN 20||substr(FROM_DATE, 8, 2)||'-06'
                  WHEN 'Jul' THEN 20||substr(FROM_DATE, 8, 2)||'-07'
                  WHEN 'Aug' THEN 20||substr(FROM_DATE, 8, 2)||'-08'
                  WHEN 'Sep' THEN 20||substr(FROM_DATE, 8, 2)||'-09'
                  WHEN 'Oct' THEN 20||substr(FROM_DATE, 8, 2)||'-10'
                  WHEN 'Nov' THEN 20||substr(FROM_DATE, 8, 2)||'-11'
                  WHEN 'Dec' THEN 20||substr(FROM_DATE, 8, 2)||'-12'
              END) AS year_month
   FROM job
   UNION ALL SELECT strftime('%Y-%m', date(year_month||'-01', '+1 month'))
   FROM dates
   WHERE year_month < date('now', '-1 month'))
SELECT year_month,
       SUM(SALARY) OVER (ROWS UNBOUNDED PRECEDING) AS CUM_SALARY
FROM
  (SELECT year_month,
          SALARY
   FROM dates
   INNER JOIN
     (SELECT SALARY,
             CASE substr(FROM_DATE, 4, 3)
                 WHEN 'Jan' THEN 20||substr(FROM_DATE, 8, 2)||'-01'
                 WHEN 'Feb' THEN 20||substr(FROM_DATE, 8, 2)||'-02'
                 WHEN 'Mar' THEN 20||substr(FROM_DATE, 8, 2)||'-03'
                 WHEN 'Apr' THEN 20||substr(FROM_DATE, 8, 2)||'-04'
                 WHEN 'May' THEN 20||substr(FROM_DATE, 8, 2)||'-05'
                 WHEN 'Jun' THEN 20||substr(FROM_DATE, 8, 2)||'-06'
                 WHEN 'Jul' THEN 20||substr(FROM_DATE, 8, 2)||'-07'
                 WHEN 'Aug' THEN 20||substr(FROM_DATE, 8, 2)||'-08'
                 WHEN 'Sep' THEN 20||substr(FROM_DATE, 8, 2)||'-09'
                 WHEN 'Oct' THEN 20||substr(FROM_DATE, 8, 2)||'-10'
                 WHEN 'Nov' THEN 20||substr(FROM_DATE, 8, 2)||'-11'
                 WHEN 'Dec' THEN 20||substr(FROM_DATE, 8, 2)||'-12'
             END AS start_date
      FROM job t1
      INNER JOIN workers t2 ON t1.WORKER_ID = t2.WORKER_ID
      WHERE FIRST_NAME = 'NIKOLA'
        AND LAST_NAME = 'TESLA') t3 ON t3.start_date <= dates.year_month);
        """, con)

,year_month,CUM_SALARY
0,2009-06,621.44
1,2009-07,1242.88
2,2009-08,1864.32
3,2009-09,2485.76
4,2009-10,3107.20
...,...,...
127,2020-01,79544.32
128,2020-02,80165.76
129,2020-03,80787.20
130,2020-04,81408.64


f. Напишите запрос, который выведет накопленную зарплату по месяцам, разбитую по позициям, за 2014 год

In [10]:
pd.read_sql_query("""
WITH RECURSIVE dates AS
  (SELECT '2014-01' AS year_month
   UNION ALL SELECT strftime('%Y-%m', date(year_month||'-01', '+1 month'))
   FROM dates
   WHERE year_month < date('2014-12-01', '-1 month'))
SELECT year_month,
       POSITION,
       SUM(SUM_SALARY) OVER (PARTITION BY POSITION
                             ORDER BY year_month) AS CUM_SALARY
FROM
  (SELECT year_month,
          POSITION,
          SUM_SALARY
   FROM dates
   INNER JOIN
     (SELECT POSITION,
             SUM(SALARY) AS SUM_SALARY,
             '2014-01' AS mon
      FROM job
      GROUP BY POSITION) t1 ON dates.year_month >= t1.mon);
      """, con)

,year_month,POSITION,CUM_SALARY
0,2014-01,AUTOMATION QA ENGINEER,2849.37
1,2014-02,AUTOMATION QA ENGINEER,5698.74
2,2014-03,AUTOMATION QA ENGINEER,8548.11
3,2014-04,AUTOMATION QA ENGINEER,11397.48
4,2014-05,AUTOMATION QA ENGINEER,14246.85
...,...,...,...
79,2014-08,WEB DESIGNER,16993.60
80,2014-09,WEB DESIGNER,19117.80
81,2014-10,WEB DESIGNER,21242.00
82,2014-11,WEB DESIGNER,23366.20
